<a href="https://colab.research.google.com/github/telmaau/conspiracy/blob/main/coref_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# packages
!pip install nltk==3.5
!pip install allennlp
!pip install --pre allennlp-models




In [ ]:
import gdown
# This will download the model from the Google Drive

!unzip /content/drive/MyDrive/colab/allennlp_models.zip #-d /content/#/usr/local/lib/python3.6/dist-packages #/usr/lib/python3


# Data

In [ ]:
# Danish data from drive
!unzip "/content/drive/MyDrive/colab/dev.danish.zip"
!unzip "/content/drive/MyDrive/colab/test.danish.zip"
!unzip "/content/drive/MyDrive/colab/train.danish.zip"

# twitter data
!unzip "/content/drive/MyDrive/colab/test2.zip" 
!unzip "/content/drive/MyDrive/colab/train2.zip" 

In [4]:
# twiconv data preparation
import random
import glob
conll_list = glob.glob("/content/test2/*_gold_conll")
conll_list2 = glob.glob("/content/train2/*_gold_conll")
conlls = conll_list+conll_list2
random.shuffle(conlls)

In [5]:
conll_list=conlls
# here each item in the list is one twitter thread

train_len = int(len(conll_list)*0.25)
dev_len = int(len(conll_list)*0.25)
test_len = int(len(conll_list)*0.50)
print(train_len)
print(dev_len)
print(test_len)

46
46
92


In [6]:
import pandas as pd

In [ ]:
# Danish pos tags
# ADJ – ADP – ADV – AUX – CCONJ – DET – INTJ – NOUN – NUM – PART – PRON – PROPN – PUNCT – SCONJ – SYM – VERB – X
mapdata = pd.read_csv("/content/drive/MyDrive/colab/en-ptb.csv", sep='\t', header=None)
mapdata

In [ ]:
# change the pos tags to match Danish data
pos_dict=mapdata.set_index(0).to_dict()
#for key in pos_dict[1].keys():
#  print(key)

pos_dict = pos_dict[1]
#pos_dict

In [10]:
train_set = []
for c in conll_list[:train_len]:
    with open(c) as f:
        test = f.readlines()
    new_test = []
    for l in test:
        if test.index(l) == 0:
            new_test.append(l)
        elif test.index(l) == len(test)-1:
            new_test.append(l)
        elif l != "\n":
            temp_line = ""
            split = l.split("\t")
            split[4] = pos_dict[split[4]]
            if split[4] == '.':
              split[4] = 'PUNCT'
            #print(split[4])
            temp_line = '\t'.join(split[:5]) + "\t-\t-\t-\t-\t*\t*\t*\t*\t*\t" + split[10] + "\n"
            new_test.append(temp_line)
        else:
            new_test.append(l)
    train_set += new_test
with open("train.conll", "w") as f:
    f.writelines(train_set)
    f.close()

In [11]:
dev_set = []
for c in conll_list[train_len:(train_len+dev_len)]:
    with open(c) as f:
        test = f.readlines()
    new_test = []
    for l in test:
        if test.index(l) == 0:
            new_test.append(l)
        elif test.index(l) == len(test)-1:
            new_test.append(l)
        elif l != "\n":
            temp_line = ""
            split = l.split("\t")
            split[4] = pos_dict[split[4]]
            if split[4] == '.':
              split[4] = 'PUNCT'
            #print(split[4])
            temp_line = '\t'.join(split[:5]) + "\t-\t-\t-\t-\t*\t*\t*\t*\t*\t" + split[10] + "\n"
            new_test.append(temp_line)
        else:
            new_test.append(l)
    dev_set += new_test
with open("dev.conll", "w") as f:
    f.writelines(dev_set)
    f.close()

In [12]:
test_set = []
for c in conll_list[(train_len+dev_len):]:
    with open(c) as f:
        test = f.readlines()
    new_test = []
    for l in test:
        if test.index(l) == 0:
            new_test.append(l)
        elif test.index(l) == len(test)-1:
            new_test.append(l)
        elif l != "\n":
            temp_line = ""
            split = l.split("\t")
            split[4] = pos_dict[split[4]]
            if split[4] == '.':
              split[4] = 'PUNCT'
            #print(split[4])
            temp_line = '\t'.join(split[:5]) + "\t-\t-\t-\t-\t*\t*\t*\t*\t*\t" + split[10] + "\n"
            new_test.append(temp_line)
        else:
            new_test.append(l)
    test_set += new_test
with open("test.conll", "w") as f:
    f.writelines(test_set)
    f.close()

# Training

In [ ]:
!rm -r /content/drive/MyDrive/colab/allennlp_folder
!allennlp train /content/drive/MyDrive/colab/config_twitter.json -s /content/drive/MyDrive/colab/allennlp_folder

In [ ]:
!allennlp evaluate /content/allennlp_folder/model.tar.gz /content/test.conll --predictions-output-file pred.jsonlines --output-file temp.txt

# Predictions

In [ ]:
from allennlp.predictors.predictor import Predictor
#predictor = Predictor.from_path("/content/drive/MyDrive/colab/mymodel.tar.gz")
predictor2 = Predictor.from_path("/content/drive/MyDrive/colab/model.tar.gz")


Use coarse to fine: True



Use coarse to fine: True


In [ ]:
sent = "Pia er glad. Hun er på vej hjem, når Maria ringer. De snakker længe, fordi Maria har mange ting at fortælle." #or any other sentence.
pred = predictor2.predict(
    document= sent
)
print(pred)
clusters = pred['clusters']
document = pred['document']
#print(document)
print(clusters)
document = pred['document']



{'top_spans': [[0, 0], [0, 2], [1, 2], [2, 2], [4, 4], [10, 11], [11, 11], [11, 12], [14, 14], [19, 19]], 'antecedent_indices': [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]], 'predicted_antecedents': [-1, -1, -1, -1, 0, -1, -1, -1, 4, 4], 'document': ['Pia', 'er', 'glad', '.', 'Hun', 'er', 'på', 'vej', 'hjem', ',', 'når', 'Maria', 'ringer', '.', 'De', 'snakker', 'længe', ',', 'fordi', 'Maria', 'har', 'mange', 'ting', 'at', 'fortælle', '.'], 'clusters': [[[0, 0], [4, 4], [14, 14], [19, 19]]]}
[[[0, 0], [4, 4], [14, 14], [19, 19]]]


In [ ]:
# Loop to show the entities in the clusters
n = 0
doc = {}
for obj in document:
    doc.update({n :  obj}) #what I'm doing here is creating a dictionary of each word with its respective index, making it easier later.
    n = n+1
    
clus_all = []
cluster = []
clus_one = {}
for i in range(0, len(clusters)):
    one_cl = clusters[i]
    for count in range(0, len(one_cl)):
        obj = one_cl[count]
        for num in range((obj[0]), (obj[1]+1)):
            for n in doc:
                if num == n:
                    cluster.append(doc[n])
    clus_all.append(cluster)
    cluster = [] 
    
print(clus_all) #shows all coreferences 

[['Pia', 'Hun', 'De', 'Maria']]
